In [1]:
# Import necessary packages
import os
import time

# Data Packages
import numpy as np
import pandas as pd

# Image Processing Packages
import cv2

# OS Screen Capture Pacakges
import mss
import mss.tools

# Concurrency Packages
import threading

In [2]:
# Define Globals
DURATION = 10 # Set testing video duration in seconds
THRESHOLD = 30 # Threshold for detecting a differencebetween frames in a recording
FRAME_RATE = 10 # Set frame rate for recording
FRAME_INTERVAL = 1/FRAME_RATE # Set frame interval for recording

In [3]:
# Create Recordings & Screenshots Directories
if not os.path.exists('./recordings'):
    os.makedirs('./recordings')

if not os.path.exists('./screenshots'):
    os.makedirs('./screenshots')

sub_directories = ['initial','compression']

for sub_directory in sub_directories:
    if not os.path.exists('./recordings/' + sub_directory):
        os.makedirs('./recordings/' + sub_directory)

In [4]:
# Define a short function for naming conventions
def name_converter(num):
    if num == 0:
        return 'total'
    else:
        return str(num)

In [5]:
# Get connected monitors, their dimensions, total combined dimensions, and take a sample screenshot of each
with mss.mss() as sct:
    # Get connected monitors
    monitors = sct.monitors
    print(monitors)

    # Take a sample screenshot of each monitor
    for i, monitor in enumerate(monitors):
        sct_img = sct.grab(monitor)
        mss.tools.to_png(sct_img.rgb, sct_img.size, output=f'./screenshots/monitor_{name_converter(i)}.png')


[{'left': -1256, 'top': -1080, 'width': 4480, 'height': 1980}, {'left': 0, 'top': 0, 'width': 1440, 'height': 900}, {'left': -1256, 'top': -1080, 'width': 1920, 'height': 1080}, {'left': 664, 'top': -1080, 'width': 2560, 'height': 1080}]


In [6]:
# Create a 4 character code of codec used to compress the frames
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

In [7]:
# Define a function to screen grab and write frames to an output video file
def record_monitor(monitor, output_file):
    output_writer = cv2.VideoWriter(output_file, fourcc, FRAME_RATE, (monitor['width'], monitor['height']))
    start_time = time.time()
    with mss.mss() as sct:
        while time.time() - start_time < DURATION:
            loop_start = time.time()

            # Grab the current frame
            sct_img = sct.grab(monitor)

            # Convert the frame to a numpy array and then to a cv2 image
            frame = np.array(sct_img)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # Write the frame to the output file
            output_writer.write(frame)

            loop_end = time.time()
            elapsed_time = loop_end - loop_start
            if elapsed_time < FRAME_INTERVAL:
                time.sleep(FRAME_INTERVAL - elapsed_time)

    output_writer.release()

In [8]:
# Record the screens
# Create VideoWriter object to save the video
# Get the recording number
recordings_dir = './recordings/initial'
recordings_count = int(len(os.listdir(recordings_dir))/len(monitors))

# Create a thread to record each monitor
threads = []
for i, monitor in enumerate(monitors):
    output_file = f'{recordings_dir}/monitor_{name_converter(i)}_{recordings_count}.mp4'
    thread = threading.Thread(target=record_monitor, args=(monitor, output_file))
    thread.start()
    threads.append(thread)

# Wait for all threads to finish
for thread in threads:
    thread.join()

recordings_count += 1

In [9]:
# Use a Threshold-Based Compression Technique
# Calculate the differences between the frames to reduce the size of the video

# Using moviepy to edit the videos
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_reader import FFMPEG_VideoReader

# Increase the analyzeduration and probesize options
FFMPEG_VideoReader.analyzeduration = 10**8
FFMPEG_VideoReader.probesize = 10**8

# Function to calculate the difference between two frames
def frame_diff(prev_frame, cur_frame, threshold):
    return np.mean(np.abs(prev_frame.astype(int) - cur_frame.astype(int))) > threshold

# Function to Process a Video
def process_video(input_file, output_file, threshold=THRESHOLD):
    # Load the video, initialize the start and end times arrays to know places we can cut
    video = mp.VideoFileClip(input_file)
    start_times = []
    end_times = []

    # Iterate over the frames in the video, keeping track of the last frame
    last_frame = None
    for t in video.iter_frames(with_times=True, dtype=int):
        time, frame = t

        # If this is the first frame, initialize the last frame
        if last_frame is not None:
            # Check to see if the difference between the current frame and the last frame is above the threshold
            if frame_diff(last_frame, frame, threshold):
                start_times.append(time)
                if len(end_times) > 0:
                    end_times[-1] = time
            else:
                if len(end_times) == 0:
                    end_times.append(time)
                else:
                    end_times[-1] = time

        last_frame = frame

    if len(start_times) > len(end_times):
        end_times.append(video.duration)

    # Create a new video with the start and end times
    print(f'Start Times: {start_times}')
    print(f'End Times: {end_times}')
    clips = [video.subclip(start, end) for start, end in zip(start_times, end_times)]
    output_video = mp.concatenate_videoclips(clips)
    output_video.write_videofile(output_file, codec="libx264")


# Function to convert a video without processing it
def convert_video(input_file, output_file):
    video = mp.VideoFileClip(input_file)
    video.write_videofile(output_file, codec="libx264")


In [10]:
# for m in range(len(monitors)):
process_video(
    f'./recordings/initial/monitor_2_0.mp4', 
    f'./recordings/compression/monitor_2_0.mp4'
)

Start Times: []
End Times: [2.1]


ValueError: max() arg is an empty sequence